In [1]:
!pip install utm

In [68]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns


# Data Science
import numpy as np
import pandas as pd

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix

# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
pc.settings.set_subscription_key('You primary key')

# Others
import requests
import rich.table
import utils
from itertools import cycle
from tqdm import tqdm
import xarray as xr
tqdm.pandas()

In [69]:
#Read Data
crop_presence_data = pd.read_csv("Crop_Location_Data.csv")
crop_presence_data

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice
...,...,...
595,"(10.013942985253381, 105.67361318732796)",Non Rice
596,"(10.01348875642372, 105.67361318732796)",Non Rice
597,"(10.013034527594062, 105.67361318732796)",Non Rice
598,"(10.012580298764401, 105.67361318732796)",Non Rice


In [70]:
import numpy as np

def get_sentinel1_data(latlong, time_slice, assets, distance=0.01):
    '''
    Returns VV and VH values for a given latitude and longitude 
    Attributes:
    latlong - A tuple with 2 elements - latitude and longitude
    time_slice - Timeframe for which the VV and VH values have to be extracted
    assets - A list of bands to be extracted
    distance - The distance to expand the bounding box around the given coordinates (default 0.01 degrees)
    '''

    latlong=latlong.replace('(','').replace(')','').replace(' ','').split(',')
    lat, lon = float(latlong[0]), float(latlong[1])
    
    bbox_of_interest = (lon - distance, lat - distance, lon + distance, lat + distance)
    time_of_interest = time_slice

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    search = catalog.search(
        collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest
    )
    items = list(search.get_all_items())
    bands_of_interest = assets
    
    data = stac_load([items[0]], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
    
    vh = data["vh"].astype("float").values
    vv = data["vv"].astype("float").values
    
    # Calculate the mean of the VV and VH values
    vh_mean = np.mean(vh)
    vv_mean = np.mean(vv)
    
    return vh_mean, vv_mean


In [72]:
import pandas as pd
from datetime import datetime, timedelta


def generate_monthly_time_slices(start_date, end_date):
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    time_slices = []

    while start_date < end_date:
        next_month_start = start_date + timedelta(days=(32 - start_date.day))
        next_month_start = next_month_start.replace(day=1)

        if next_month_start > end_date:
            next_month_start = end_date

        time_slice = f"{start_date.date()}/{next_month_start.date()}"
        time_slices.append(time_slice)
        start_date = next_month_start

    return time_slices

# Function call to generate monthly time slices for the specified date range
time_slices = generate_monthly_time_slices("2022-01-01", "2023-01-01")
assets = ['vh', 'vv']

vh_vv = []

for time_slice in time_slices:
    vh_vv_month = []
    
    for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
        vh_vv_month.append(get_sentinel1_data(coordinates, time_slice, assets))

    vh_vv.append(vh_vv_month)

# Flatten the list of lists and convert it to a DataFrame
vh_vv_flat = [item for sublist in vh_vv for item in sublist]
vh_vv_data = pd.DataFrame(vh_vv_flat, columns=['vh', 'vv'])


100%|██████████| 600/600 [03:05<00:00,  3.23it/s]  


In [85]:
vh_vv_data.shape

(7200, 3)

In [75]:
# Calculate RVI
dop = (vh_vv_data.vv / (vh_vv_data.vv + vh_vv_data.vh))
m = 1 - dop
vh_vv_data['rvi'] = (np.sqrt(dop))*((4*vh_vv_data.vh)/(vh_vv_data.vv + vh_vv_data.vh))

In [77]:
field_ids = np.repeat(np.arange(0, 600), 12)
vh_vv_data['field'] = field_ids

In [79]:
month_numbers = np.tile(np.arange(1, 13), 600)
vh_vv_data['month'] = month_numbers

In [82]:
vh_vv_data = vh_vv_data.set_index(['field', 'month'], inplace=True)

In [86]:
vh_vv_data.head()

vh        vv       rvi
field month                              
0     1      0.032352  0.182373  0.555414
      2      0.040305  0.224249  0.561068
      3      0.032408  0.182790  0.555172
      4      0.032328  0.182611  0.554534
      5      0.032389  0.191094  0.536061

In [ ]:
import seaborn as sns

# Setting the dimensions of the plot
fig, ax = plt.subplots(figsize=(20, 10))

# Create the line plot
sns.lineplot(data=vh_vv_data, x="month", y="rvi", hue="field").set(title='RVI values - Training dataset')

# Save the plot to a file
fig.savefig("grvi_sa.png")

# Show the plot
plt.show()

In [87]:
import pandas as pd

vh_vv_data.to_csv('s1_data.csv', index=True)